: 

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import re
import random

In [2]:
Books = pd.read_csv("Books.csv", low_memory=False)
Ratings_no_zero = pd.read_csv("Filtered_Ratings_NoZero.csv", low_memory=False)
users = pd.read_csv("Users.csv", low_memory=False)


In [3]:
Books_Ratings=Books.merge(Ratings_no_zero,on="ISBN")
Books_Ratings.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,User-ID,Book-Rating
0,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,8,5
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,11676,8
2,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,67544,8
3,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,116866,9
4,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,123629,9


In [10]:
Books_Ratings_df=Books_Ratings.copy()
Books_Ratings_df.dropna(inplace=True)
Books_Ratings_df.reset_index(drop=True,inplace=True)
Books_Ratings_df.drop(columns=["ISBN","Year-Of-Publication","Image-URL-S","Image-URL-M", "Publisher", "Image-URL-L"],axis=1,inplace=True)
Books_Ratings_df.drop(index=Books_Ratings_df[Books_Ratings_df["Book-Rating"]==0].index,inplace=True)
Books_Ratings_df["Book-Title"] = Books_Ratings_df["Book-Title"].apply(lambda x : x.strip())
# Books_Ratings_df["Book-Title"]=Books_Ratings_df["Book-Title"].apply(lambda x: re.sub("[\W_]+"," ",x).strip())
Books_Ratings_df.head(100)

,Book-Title,Book-Author,User-ID,Book-Rating
0,Clara Callan,Richard Bruce Wright,8,5
1,Clara Callan,Richard Bruce Wright,11676,8
2,Clara Callan,Richard Bruce Wright,67544,8
3,Clara Callan,Richard Bruce Wright,116866,9
4,Clara Callan,Richard Bruce Wright,123629,9
...,...,...,...,...
95,The Testament,John Grisham,85534,7
96,The Testament,John Grisham,86969,8
97,The Testament,John Grisham,88006,4
98,The Testament,John Grisham,93055,7


In [11]:
# 책 평가 200번 초과한 사람들 -> users_200_df
users_200_df=Books_Ratings_df[Books_Ratings_df['User-ID'].map(Books_Ratings_df['User-ID'].value_counts()) > 200]
users_pv=users_200_df.pivot_table(index=["User-ID"],columns=["Book-Title"],values="Book-Rating")
users_pv.fillna(0,inplace=True)

In [12]:
# 유저의 최고 평점 책 5권 골라 리턴
def best_5_books(id):
    
    best5=users_200_df[users_200_df["User-ID"]==id].sort_values(["Book-Rating"],ascending=False)[0:5]
    return best5

In [13]:
best_5_books(269566)["Book-Title"]

277587                               King Lear (3rd Series)
277813                                        Sydney's Star
278367    Cyndy Szekeres' Teeny Mouse Counts Herself (Go...
53899     The Tragedy of Hamlet Prince of Denmark (Signe...
278088    Little Bear Counts His Favorite Things (Golden...
Name: Book-Title, dtype: object

In [ ]:
# centered cosine
# def cosine_similarity_non_zero(matrix):
#     num_rows = matrix.shape[0]
#     similarity_matrix = np.zeros((num_rows, num_rows))

#     for i in range(num_rows):
#         for j in range(i, num_rows):
#             # 두 벡터에서 0이 아닌 요소의 인덱스를 찾음
#             non_zero_indices_i = matrix[i].nonzero()
#             non_zero_indices_j = matrix[j].nonzero()
#             common_indices = np.intersect1d(non_zero_indices_i, non_zero_indices_j)

#             if len(common_indices) > 0:
#                 # 공통 인덱스에 해당하는 요소만 추출하여 코사인 유사도 계산
#                 vector_i = matrix[i, common_indices]
#                 vector_j = matrix[j, common_indices]
#                 cos_sim = np.dot(vector_i, vector_j) / (np.linalg.norm(vector_i) * np.linalg.norm(vector_j))
                
#                 similarity_matrix[i, j] = cos_sim
#                 similarity_matrix[j, i] = cos_sim  # 대칭성을 유지
#             else:
#                 similarity_matrix[i, j] = 0
#                 similarity_matrix[j, i] = 0

#     return similarity_matrix

: 

In [14]:
# cosine similarity 활용해 가장 유사한 유저 5명 찾아 user id 리턴
def most_similar_5users(users_200_df, id):
    if id not in users_200_df["User-ID"].values:
        print("---일치하는 유저 없습니다---")
        
        
    else:
        idx=np.where(users_pv.index == id)[0][0]
        similarity=cosine_similarity(users_pv)
        
        similar_users=list(enumerate(similarity[idx]))
        
        # 0번째 id는 제외(자기 자신임)
        similar_users = sorted(similar_users,key = lambda x:x[1],reverse=True)[1:6]
        
          
        most_sim_users=[]
        

        for i in similar_users:
                all_ids=Books_Ratings_df[Books_Ratings_df["User-ID"]==users_pv.index[i[0]]]
                most_sim_users.extend(list(all_ids.drop_duplicates("User-ID")["User-ID"].values))
                
        
        
    return most_sim_users

In [16]:
my_id = random.choice(users_200_df["User-ID"].values)
most_sim_users = most_similar_5users(users_200_df,my_id)
print("ME:", my_id)
print("FIVE SIMILAR USERS:", most_sim_users)



ME: 235105
FIVE SIMILAR USERS: [98391, 11676, 114988, 158295, 142524]


In [17]:
# user(userid와 유사한 유저 목록) 와 user_id가 평가한 책들에서 
# userid가 아직 읽지 않은 책 상위 5권 리턴
def recommend_5_books(users_200_df, user, user_id):
    me=users_200_df[users_200_df["User-ID"]==user_id]
    recom_5_books=[]
    user=list(user)

    for i in user:
        sim_users=users_200_df[(users_200_df["User-ID"]==i)]

        # 유사한 유저 중 내가 읽은거 제외
        books=sim_users.loc[~sim_users["Book-Title"].isin(me["Book-Title"]),:]

        # rating 높은 순서대로 5권 return
        books=books.sort_values(["Book-Rating"],ascending=False)[0:5]
        recom_5_books.extend(books["Book-Title"].values)
        
    return recom_5_books[0:5]

In [30]:
my_id = random.choice(users_200_df["User-ID"].values)
my_best_5_books_df = pd.DataFrame(best_5_books(my_id))
my_best_5_books = best_5_books(my_id)

print("*** ME: {} ***".format(my_id))
print("=======================================================================================")
print()

# 내가 가장 좋아하는 책 5권의 평균 평점 
my_best_books = []
for i in range(5):
    title = my_best_5_books_df["Book-Title"].tolist()[i]
    rating = round(users_200_df[users_200_df["Book-Title"] == title]["Book-Rating"].mean(), 1)
    my_best_books.append((title, rating))

# 비슷한 유저 찾아서 나만 안읽은 책 중 평점 높은 순서로 5권
sim_users = most_similar_5users(users_200_df, my_id)
recom_books = recommend_5_books(users_200_df, sim_users, my_id)
recommended_books = []

for i in range(5):
    title = recom_books[i]
    rating = round(users_200_df[users_200_df["Book-Title"] == title]["Book-Rating"].mean(), 1)
    recommended_books.append((title, rating))

# 출력

print("*** 내가 가장 좋아하는 책 5권의 평균 평점 ***")
print()
for book, rating in my_best_books:
    print(f"* 책 이름 * {book}, * 평점 * {rating}")
print("=======================================================================================")
print()

print("*** 추천할 책들 ***")
print()
for book, rating in recommended_books:
    print(f"* 책 이름 * {book}, * 평점 * {rating}")   
print("=======================================================================================")
print()

print("*** 너와 유사한 취향을 가진 유저 5명과 각 유저의 책 5권씩 소개해줄게 ***")
print()
for id in sim_users:
    print("User_ID:", id)
    
    for book in best_5_books(id)["Book-Title"]:
        print("-",book)
    print("--------------------------------------------------------------------------------")


*** ME: 63714 ***

*** 내가 가장 좋아하는 책 5권의 평균 평점 ***

* 책 이름 * 20,000 Leagues Under the Sea (Wordsworth Collection), * 평점 * 10.0
* 책 이름 * Hms Beagle: The Story of Darwin's Ship, * 평점 * 10.0
* 책 이름 * Fossils, minerals, and rocks: Collection and preservation, * 평점 * 10.0
* 책 이름 * The Dinosaur Questions, * 평점 * 10.0
* 책 이름 * Man the tool-maker (British Museum), * 평점 * 10.0

*** 추천할 책들 ***

* 책 이름 * Guillaume Chequespierre, * 평점 * 10.0
* 책 이름 * Gifts of Nature: An American Sampler (An American sampler), * 평점 * 10.0
* 책 이름 * Native American Cross Stitch, * 평점 * 10.0
* 책 이름 * Donna Kooler's 555 Fabulous Cross-Stitch Patterns, * 평점 * 10.0
* 책 이름 * Carousel Horses in Cross-Stitch: Beautiful Projects for Every Month of the Year, * 평점 * 10.0

*** 너와 유사한 취향을 가진 유저 5명과 각 유저의 책 5권씩 소개해줄게 ***

User_ID: 56399
- The Catalog of Lost Books: An Annotated and Seriously Addled Collection of Great Books That Should Have Been Written but Never Were
- Carpe Jugulum (Discworld Novels (Paperback))
- Gifts of Natur

: 